### Summarising, Aggregating, and Grouping data in Python Pandas

In [1]:
import pandas as pd
import dateutil # the date column can be parsed using the extremely handy dateutil library

In [2]:
#load data into DataFrame
data = pd.read_csv('phone_data.csv')
data.head(10)

,index,date,duration,item,month,network,network_type
0,0,15/10/14 06:58,34.429,data,2014-11,data,data
1,1,15/10/14 06:58,13.000,call,2014-11,Vodafone,mobile
2,2,15/10/14 14:46,23.000,call,2014-11,Meteor,mobile
3,3,15/10/14 14:48,4.000,call,2014-11,Tesco,mobile
4,4,15/10/14 17:27,4.000,call,2014-11,Tesco,mobile
5,5,15/10/14 18:55,4.000,call,2014-11,Tesco,mobile
6,6,16/10/14 06:58,34.429,data,2014-11,data,data
7,7,16/10/14 15:01,602.000,call,2014-11,Three,mobile
8,8,16/10/14 15:12,1050.000,call,2014-11,Three,mobile
9,9,16/10/14 15:30,19.000,call,2014-11,voicemail,voicemail


The main columns in the file are:

date: The date and time of the entry  
duration: The duration (in seconds) for each call, the amount of data (in MB) for each data entry, and the number of texts sent (usually 1) for each sms entry.  
item: A description of the event occurring – can be one of call, sms, or data.  
month: The billing month that each entry belongs to – of form ‘YYYY-MM’.  
network: The mobile network that was called/texted for each entry.  
network_type: Whether the number being called was a mobile, international (‘world’), voicemail, landline, or other (‘special’) number.

In [3]:
# convert date from string to date times
data['date'] = data['date'].apply(dateutil.parser.parse, dayfirst=True)

In [4]:
data.head()

,index,date,duration,item,month,network,network_type
0,0,2014-10-15 06:58:00,34.429,data,2014-11,data,data
1,1,2014-10-15 06:58:00,13.000,call,2014-11,Vodafone,mobile
2,2,2014-10-15 14:46:00,23.000,call,2014-11,Meteor,mobile
3,3,2014-10-15 14:48:00,4.000,call,2014-11,Tesco,mobile
4,4,2014-10-15 17:27:00,4.000,call,2014-11,Tesco,mobile


### Summarizing the DataFrame

In [5]:
# How many rows in the dataset
data['item'].count()

830

In [6]:
# What was the longest phone call / data entry?
data['duration'].max()

10528.0

In [7]:
# How many seconds of phone calls are recorded in total?
data['duration'][data['item']=='call'].sum()

92321.0

In [8]:
# How many entries are there for each month?
data['month'].value_counts()

2014-11    230
2015-01    205
2014-12    157
2015-02    137
2015-03    101
Name: month, dtype: int64

In [9]:
# Number of non-null unique network entries
data['network'].nunique()

9

## Summarizing Groups in the DataFrame

In [10]:
data.groupby(['month']).groups.keys()

['2014-12', '2014-11', '2015-02', '2015-03', '2015-01']

In [11]:
len(data.groupby(['month']).groups['2014-11'])

230

In [12]:
# Get the first entry for each month
data.groupby('month').first()

,index,date,duration,item,network,network_type
month,,,,,,
2014-11,0,2014-10-15 06:58:00,34.429,data,data,data
2014-12,228,2014-11-13 06:58:00,34.429,data,data,data
2015-01,381,2014-12-13 06:58:00,34.429,data,data,data
2015-02,577,2015-01-13 06:58:00,34.429,data,data,data
2015-03,729,2015-02-12 20:15:00,69.000,call,landline,landline


In [13]:
# Get the sum of the durations per month
data.groupby('month')['duration'].sum()

month
2014-11    26639.441
2014-12    14641.870
2015-01    18223.299
2015-02    15522.299
2015-03    22750.441
Name: duration, dtype: float64

In [14]:
# Get the number of dates / entries in each month
data.groupby('month')['date'].count()

month
2014-11    230
2014-12    157
2015-01    205
2015-02    137
2015-03    101
Name: date, dtype: int64

In [15]:
# What is the sum of durations, for calls only, to each network
data[data['item']=='call'].groupby('network')['duration'].sum()

network
Meteor        7200
Tesco        13828
Three        36464
Vodafone     14621
landline     18433
voicemail     1775
Name: duration, dtype: float64

In [16]:
# How many calls, sms, and data entries are in each month?
data.groupby(['month','item'])['date'].count()

month    item
2014-11  call    107
         data     29
         sms      94
2014-12  call     79
         data     30
         sms      48
2015-01  call     88
         data     31
         sms      86
2015-02  call     67
         data     31
         sms      39
2015-03  call     47
         data     29
         sms      25
Name: date, dtype: int64

In [17]:
# How many calls, texts, and data are sent per month, split by network_type?
data.groupby(['month','network_type'])['date'].count()

month    network_type
2014-11  data             29
         landline          5
         mobile          189
         special           1
         voicemail         6
2014-12  data             30
         landline          7
         mobile          108
         voicemail         8
         world             4
2015-01  data             31
         landline         11
         mobile          160
         voicemail         3
2015-02  data             31
         landline          8
         mobile           90
         special           2
         voicemail         6
2015-03  data             29
         landline         11
         mobile           54
         voicemail         4
         world             3
Name: date, dtype: int64

## Multiple Statistics per Group

In [18]:
# Group the data frame by month and item and extract a number of stats from each group
aggregations = {
    'duration':'sum',
    'date': lambda x: max(x)
}
data.groupby('month').agg(aggregations)

,duration,date
month,,
2014-11,26639.441,2014-11-13 22:31:00
2014-12,14641.870,2014-12-14 19:54:00
2015-01,18223.299,2015-01-14 23:36:00
2015-02,15522.299,2015-02-12 06:58:00
2015-03,22750.441,2015-03-14 00:16:00


In [19]:
# Define the aggregation calculations
aggregations2 = {
    'duration': { # work on 'duration' column
        'total_duration': 'sum',
        'average_duration': 'mean',
        'num_calls': 'count'
    },
    'date': { # work on 'date' column
        'max_date': 'max',
        'min_date': 'min',
        'num_days': 'count'
    },
    'network': ['count','max']
}
data[data['item']=='call'].groupby('month').agg(aggregations2)

duration                                         date  \
        average_duration num_calls total_duration            max_date   
month                                                                   
2014-11       238.757009       107          25547 2014-11-12 19:01:00   
2014-12       171.658228        79          13561 2014-12-14 19:54:00   
2015-01       193.977273        88          17070 2015-01-14 20:47:00   
2015-02       215.164179        67          14416 2015-02-09 17:54:00   
2015-03       462.276596        47          21727 2015-03-04 12:29:00   

                                     network             
        num_days            min_date   count        max  
month                                                    
2014-11      107 2014-10-15 06:58:00     107  voicemail  
2014-12       79 2014-11-14 17:24:00      79  voicemail  
2015-01       88 2014-12-15 20:03:00      88  voicemail  
2015-02       67 2015-01-15 10:36:00      67  voicemail  
2015-03       47 2015-02-12 20:15:00      47  voicemail